In [6]:
import os
import shutil

### Converting data to nnU-Net format
The following code was used to convert the images in the training/validation/test set to a format compatible with the nnU-Net. This mostly includes renaming the images, reorganizing the folders and zipping the NIfTY images to go from .nii to .nii.gz.

The final format looks like this:

    nnUNet_raw_data_base/nnUNet_raw_data/

        ├── Task555_MultipleSclerosis

            ├── Task555_MultipleSclerosis/

                ├── dataset.json

                ├── imagesTr (Training images)
                    ├── MS_001_0000.nii.gz (0000 = FLAIR)
                    ├── MS_001_0001.nii.gz (0001 = T1)
                    ├──  ...
                    ├── MS_070_0000.nii.gz (Between 001-070 as there are 70 images in the training set)
                    ├── MS_070_0001.nii.gz 

                ├── imagesTs (Testing images)
                    ├── MS_071_0000.nii.gz
                    ├── MS_071_0001.nii.gz
                    ├──  ...
                    ├── MS_079_0000.nii.gz (Between 071 and 079 as there are 8 images in the test/validation set)
                    ├── MS_079_0001.nii.gz

                └── labelsTr (Lesion masks to use while training, numbered to correspond to subject in training set)
                    ├── MS_001.nii.gz
                    ├── ...
                    ├── MS_070.nii.gz



### Converting images
The code copies the T1 and FLAIR images from their original folder into their destination folder looking like the ones above. The training images are moved into imagesTr, the lesion masks belonging to the T1 and FLAIR training images go into labelsTr and testing images into imagesTs.

The same code is used to copy, move and rename the training, validation and testing images. All that need to change is the path to the original folder, and path to the destination folder. The code below show how it was used to convert the training images.

The number 001-079 shown above are arbitrary. Any number is usable as long as it doesnt overlap, this code uses the Icometrix number already assigned to each subject so that the training images and their lesion masks easily correspond.

In [1]:
#Getting the original path and destination path
#These variables have to be changed when wanting to convert validation and testset images
original_path = "/home/maarhus/Documents/Master/data_reoriented/MS_DATA_TRAIN"
destination_path = "/home/maarhus/Documents/Master/nnUNET_data/directory_env/nnUnetFrame/dataset/nnUnet_raw/nnUNet_raw_data/Task555_MultipleSclerosis/imagesTr"

#Adding all subjects from the chosen set into a list
#Training set = 70, validation = 8, test = 8
all_folder_list = []
for path, dirs, files in os.walk(original_path):
    if len(dirs) == 0:
        all_folder_list.append(os.path.basename(path))


In [16]:
#Iterating through every subject in the chosen set
for folder in all_folder_list:

    #Getting path to T1 and FLAIR images
    t1_path = f"{original_path}/{folder}/T1.nii"
    flair_path = f"{original_path}/{folder}/FLAIR.nii"

    #Copy T1 image into destication and renaming it to fit nnU-Net format
    shutil.copy2(t1_path,destination_path)
    old_t1_name = f"{destination_path}/T1.nii"
    new_t1_name = f"{destination_path}/MS_{folder}_0001.nii"
    os.rename(old_t1_name, new_t1_name)

    #Copy FLAIR image into destication and renaming it to fit nnU-Net format
    shutil.copy2(flair_path,destination_path)
    old_flair_name = f"{destination_path}/FLAIR.nii"
    new_flair_name = f"{destination_path}/MS_{folder}_0000.nii"
    os.rename(old_flair_name, new_flair_name)

    #Zipping the T1 and FLAIR to fit nnU-Net format
    !cd "{destination_path}" && gzip "MS_{folder}_0001.nii"
    !cd "{destination_path}" && gzip "MS_{folder}_0000.nii"


#### Converting lesion masks
The same is done for the lesion masks to convert them into the nnU-Net format. The only difference is the name given and the copying and renaming only have to happen once as it is only one image per subject.

In [1]:
#Getting the original path and destination path
labels_folder_path = "/home/maarhus/Documents/Master/data_reoriented/MS_DATA_TRAIN"
labels_destination_path = "/home/maarhus/Documents/Master/nnUNET_data/directory_env/nnUnetFrame/dataset/nnUnet_raw/nnUNet_raw_data/Task555_MultipleSclerosis/labelsTr"

#Adding all subjects from the chosen set into a list
all_labels_list = []
for path, dirs, files in os.walk(labels_folder_path):
    if len(dirs) == 0:
        all_labels_list.append(os.path.basename(path))


In [20]:
for folder in all_labels_list:
    label_path = f"{labels_folder_path}/{folder}/lesions_binarised.nii"

    #Copy and rename label
    shutil.copy2(label_path,labels_destination_path)
    old_label_name = f"{labels_destination_path}/lesions_binarised.nii"
    new_label_name = f"{labels_destination_path}/MS_{folder}.nii"
    os.rename(old_label_name, new_label_name)

    !cd "{labels_destination_path}" && gzip "MS_{folder}.nii"
